# Data Cleaning and Preparation

In [21]:
import numpy as np
import pandas as pd


## Data Transformation (continue)
So far in this lesson we’ve been concerned with rearranging data. Filtering, cleaning,
and other transformations are another class of important operations.

### Replacing Values
Filling in missing data with the `fillna` method is a special case of more general value
replacement. As you’ve already seen, `map` can be used to modify a subset of values in
an object but `replace` provides a simpler and more flexible way to do so. 

Let’s consider this Series

In [22]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [23]:
# replace the value of -999 with np.nan
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [24]:
# do the same with map function
data.map(lambda x: x if x != -999 else np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

If you want to replace multiple values at once, you instead pass a list and then the
substitute value

In [25]:
# replace both the values of -999 and -1000 with np.nan
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, pass a list of substitutes, or a mapping dict 


In [26]:
# replace both the values of -999 and -1000 with np.nan and 0 respectivly
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [27]:
# replace both the values of -999 and -1000 with np.nan and 0 respectivly
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes
Like values in a Series, axis labels can be similarly transformed by a function or mapping 
of some form to produce new, differently labeled objects. You can also modify
the axes in-place without creating a new data structure.

In [28]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


Like a Series, the axis indexes have a map method

In [36]:
# convert the indices to upper case
data.index = data.index.map(str.upper)
# x = "Ahmed"
# x.upper()
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


If you want to create a transformed version of a dataset without modifying the original, a useful method is `rename`

In [37]:
# convert the indices and column names to upper case using rename method
data.rename(index=str.upper, columns=str.upper)

,ONE,TWO,THREE,FOUR
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [39]:
# rename specific index and column
data.rename(index={"OHIO": "RIYADH"}, columns={"one": "uno"})


,uno,two,three,four
RIYADH,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [40]:
# rename index in-place
data.rename(index={"OHIO": "RIYADH"}, columns={"one": "uno"}, inplace=True)
data

,uno,two,three,four
RIYADH,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


## Independent Practice
using the `coffee-preferences.csv` data set
- show the duplicated rows
- show the names of customers who rated all coffee
- show the names of the coffees that are rated by all customers
- which coffee got the highest rate in average
- replace the missing value by zero
- convert names to upper case
- convert the rate to a percentage 

### Discretization and Binning
Continuous data is often discretized or otherwise separated into “bins” for analysis.
Suppose you have data about a group of people in a study, and you want to group
them into discrete age buckets

In [42]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To
do so, you have to use `cut`, a function in pandas

In [43]:
# divide the ages to 4 classes using the bins
bins = [18, 25, 35, 60, 100]
age_groups = pd.cut(ages, bins)
age_groups

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object pandas returns is a special **Categorical object**. The output you see
describes the bins computed by `pandas.cut`. 

You can treat it like an **array of strings** indicating the bin name; 

internally it contains a **categories array** specifying the distinct category names along with a labeling for the ages data in the **codes attribute**:

In [44]:
# print the ages codes
age_groups.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [45]:
# print the distinct categories
age_groups.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [46]:
# print the number of occurrances of each category 
age_groups.value_counts()

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

**Note** that `cats.value_counts()` are the bin counts for the result of `pandas.cut`.
Consistent with mathematical notation for intervals, a **parenthesis** means that the side
is **open**, while the **square bracket** means it is **closed (inclusive)**. 

You can change which side is closed by passing `right=False`:

In [47]:
# categorize the ages with right open intervals
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

You can also pass your own bin names by passing a list or array to the labels option:

In [48]:
# categorize the ages with the labels below
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
age_groups = pd.cut(ages, bins, labels=group_names)
age_groups

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [49]:
age_groups.value_counts()

Youth         5
YoungAdult    3
MiddleAged    3
Senior        1
dtype: int64

If you pass an integer number of bins to `cut` instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data.

In [50]:
data = np.random.randint(0, 20, 12)
data

array([ 2, 11, 17, 14,  8,  8, 19, 16,  9, 14, 16, 10])

In [53]:
# divide the data to 4 groups
print(ages)
age_groups = pd.cut(ages, 4, precision=0)
age_groups

[20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]


[(20.0, 30.0], (20.0, 30.0], (20.0, 30.0], (20.0, 30.0], (20.0, 30.0], ..., (30.0, 40.0], (51.0, 61.0], (40.0, 51.0], (40.0, 51.0], (30.0, 40.0]]
Length: 12
Categories (4, interval[float64, right]): [(20.0, 30.0] < (30.0, 40.0] < (40.0, 51.0] < (51.0, 61.0]]

In [54]:
# count the number of values in each group
age_groups.value_counts()

(20.0, 30.0]    6
(30.0, 40.0]    3
(40.0, 51.0]    2
(51.0, 61.0]    1
dtype: int64

A closely related function, `qcut`, bins the data based on sample quantiles. 

Depending on the distribution of the data, using `cut` will not usually result in each bin having the
same number of data points. 

Since `qcut` uses sample quantiles instead, by definition you will obtain **roughly equal-size** bins:

In [58]:
data = np.random.randint(0, 100, 1000)  
# Cut into quartiles and count the values
groups = pd.qcut(data, 4, precision=5)
groups.value_counts()

(-1e-05, 24.0]    259
(24.0, 48.0]      248
(48.0, 73.0]      245
(73.0, 99.0]      248
dtype: int64